In [61]:
import os
import pandas as pd
from torchvision import transforms
from PIL import Image
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import random_split, Dataset, DataLoader
import numpy as np
import json

In [55]:
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(args['size']),
    transforms.Normalize(mean=args['normalization_mean'], std = args['normalization_std'])
])
X = []
for file in os.listdir('../data/test'):
    X.append(torch.unsqueeze(convert_tensor(resizer(Image.open(path+'/'+file).convert('RGB'))),0))

file = pd.DataFrame(os.listdir('../data/test'), columns = ['file'])

In [5]:
path = '../data/test'
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((324,324)),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std = (0.229, 0.224, 0.225))
])
X = []
for file in os.listdir('../data/test'):
    X.append(torch.unsqueeze(test_transform(Image.open(path+'/'+file).convert('RGB')),0))
torch.cat(X,dim=0).shape

C:\Users\Owen\AppData\Roaming\Python\Python38\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


torch.Size([794, 3, 324, 324])

In [71]:
model = load_model(model_name = 'resnet50')
model.load_state_dict(torch.load('../models/'+os.listdir('../models')[0])['net'])
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [75]:
torch.load('../models/'+os.listdir('../models')[0])['acc']

0.9698660714285714

In [13]:
test_dataset = PlantTestDataset(torch.cat(X,dim=0))

In [15]:
test_loader = DataLoader(test_dataset, batch_size = 8,shuffle=False)

In [17]:
device = torch.device("cuda:{}".format(0) if torch.cuda.is_available() else "cpu")

In [41]:
def test(test_loader, model, device=device):
    model.to(device)
    predictions = []
    with torch.no_grad():
        for image in test_loader:
            image = image.to(device)
            y_pred = model(image)
            y_pred_labels = y_pred.argmax(dim=1)
            y_pred_labels = y_pred_labels.cpu().numpy()
            predictions.append(y_pred_labels)
    return predictions

In [50]:
predictions = np.concatenate(test(test_loader, model)).ravel()

In [65]:
idx_to_class

{'0': 'Black-grass',
 '1': 'Charlock',
 '2': 'Cleavers',
 '3': 'Common Chickweed',
 '4': 'Common wheat',
 '5': 'Fat Hen',
 '6': 'Loose Silky-bent',
 '7': 'Maize',
 '8': 'Scentless Mayweed',
 '9': 'Shepherds Purse',
 '10': 'Small-flowered Cranesbill',
 '11': 'Sugar beet'}

In [66]:
file['species'] = predictions

with open('../data/processed/idx_to_class.txt', "r") as fp:
    idx_to_class = json.load(fp)     
    

In [69]:
file['species'] = file['species'].astype('str')
file = file.replace({"species": idx_to_class})
file

,file,species
0,0021e90e4.png,Small-flowered Cranesbill
1,003d61042.png,Fat Hen
2,007b3da8b.png,Sugar beet
3,0086a6340.png,Common Chickweed
4,00c47e980.png,Sugar beet
...,...,...
789,fea355851.png,Loose Silky-bent
790,fea3da57c.png,Sugar beet
791,fef2ade8c.png,Sugar beet
792,ff65bc002.png,Charlock


In [76]:
if os.path.exists("../result"):
    file.to_csv("../result/1.csv", index=False)
else:
    os.makedirs("../result")
    file.to_csv("../result/1.csv", index=False)


In [2]:
def load_model(model_name = 'resnet50', freeze_counter=7):
    if model_name == 'resnet50':
        model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

    if model_name == 'resnet101':
        model = models.resnet101(weights=models.ResNet101_Weights.DEFAULT)
    
    elif model_name == 'inception':
        model = models.inception_v3(pretrained=True)

    for c, child in enumerate(model.children()):
        if c<freeze_counter:
            for param in child.parameters():
                param.requires_grad = False

    num_ftrs = model.fc.in_features
    
    model.fc = torch.nn.Sequential(
        torch.nn.Linear(num_ftrs, 256),
        torch.nn.ReLU(),
        torch.nn.Dropout(0.5),
        torch.nn.Linear(256, 12)
    )
    '''
    model.fc = torch.nn.Sequential(
        torch.nn.Linear(num_ftrs, 1200),
        torch.nn.BatchNorm1d(1200),
        torch.nn.ReLU(),
        torch.nn.Dropout(0.2),
        torch.nn.Linear(1200, 12)
    )
    '''

    return model


In [12]:
class PlantTestDataset(Dataset):
    def __init__(self, X):
        self.X = X
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        item = self.X[idx]
        return item